In [1]:
import matplotlib
matplotlib.use('Agg')
import numpy as np
import glob
import pypianoroll as ppr
import time
import music21
import os
import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
import torch.utils.data as data
from utils.utilsPreprocessing import *
import time
#np.set_printoptions(threshold=np.inf)
#torch.set_printoptions(threshold=50000)

In [2]:
###HYPERPARAMETERS######################
epochs = 500
batch_size = 7
learning_rate = 1e-3
log_interval = 1
hidden_size = 1024
num_layers = 1
num_classes = 61
validate = False #play/show a random sample after each epoch

# Beat resolution
If beat resolution = 1 --> 1 tick = 1/4 note<br>
If beat resolution = 2 --> 1 tick = 1/8 note<br>
If beat resolution = 4 --> 1 tick = 1/16 note<br>
If beat resolution = 8 --> 1 tick = 1/32 note<br>
...



In [3]:
beat_resolution = 4

In [4]:
pathToFiles = "/media/EXTHD/niciData/Datasets/Nottingham/"
midiDatasetTrain = createDatasetLSTM(pathToFiles + "train/*.mid", beat_res = beat_resolution,
                                    force_length=True, force_value=16)
midiDatasetTrain.setMaxLength()
##TO DO
#midiDatasetTrain.setMeanLength()

midiDatasetTest = createDatasetLSTM(pathToFiles + "test/*.mid", beat_res = beat_resolution,
                                   force_length=True, force_value=16)
midiDatasetTest.setMaxLength()

midiDatasetVal = createDatasetLSTM(pathToFiles + "valid/*.mid",
                                  force_length=True, force_value=16)
midiDatasetVal.setMaxLength()

train_loader = torch.utils.data.DataLoader(midiDatasetTrain, batch_size=batch_size, shuffle=False, drop_last=True)
test_loader = torch.utils.data.DataLoader(midiDatasetTest, batch_size=batch_size, shuffle=False, drop_last=True)
val_loader = torch.utils.data.DataLoader(midiDatasetVal, batch_size=1, shuffle=True, drop_last=True)

Longest sequences contains 16 ticks
Longest sequences contains 16 ticks
Longest sequences contains 16 ticks


In [5]:
print("\nThere are {} songs in the training set\n".format(len(midiDatasetTrain)))
print("There are {} songs in the test set\n".format(len(midiDatasetTest)))
print("There are {} songs in the validation set\n".format(len(midiDatasetVal)))


There are 694 songs in the training set

There are 170 songs in the test set

There are 173 songs in the validation set



In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [7]:
class LSTM_notewise(nn.Module):
    def __init__(self, hidden_size=400, num_layers=2, batch_size=1):
        super(LSTM_notewise, self).__init__()
        
        self.input_size = 61
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.batch_size = batch_size
        self.num_classes = 61

        
        self.i2h = nn.Linear(self.input_size, self.hidden_size)
        #self.batch_norm = nn.BatchNorm1d(self.hidden_size)
        self.lstm = nn.LSTM(self.hidden_size, self.hidden_size,
                            self.num_layers, batch_first=True,
                            dropout=0)
        self.h2o = nn.Linear(self.hidden_size, self.num_classes)
        
    def initState(self):
        state = torch.zeros(self.num_layers,
                            self.batch_size, 
                            self.hidden_size).double().to(device)
        return state
    
    def forward(self, input, seq_lengths):
        h_t = self.initState()
        c_t = self.initState()
        
        embedded_notes = self.i2h(input)
        #embedded_notes = self.batch_norm(embedded_notes)
        embedded_notes = torch.nn.utils.rnn.pack_padded_sequence(embedded_notes, 
                                                                 seq_lengths, 
                                                                 batch_first=True)    
        out, (h_t, c_t) = self.lstm(embedded_notes, (h_t, c_t))
        out, out_lengths = torch.nn.utils.rnn.pad_packed_sequence(out, batch_first=True)
        out = self.h2o(out)
        neg_out = (1-out)
        
        #print('out', out)
        #print('neg_out', neg_out)
        out = torch.stack((out, neg_out), dim=3)
        out = out.view(-1, 2)

        
        return out

In [8]:
model = LSTM_notewise(hidden_size=hidden_size, num_layers=num_layers, 
                      batch_size=batch_size).double().to(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

#initialize lstm weights
from torch.nn import init
init_range = 1.
for layer_p in model.lstm._all_weights:
    for p in layer_p:
        if 'weight' in p:
            init.xavier_uniform_(model.lstm.__getattr__(p), gain=10)

In [9]:
#load model
#from loadModel import loadStateDict
#pathToModel = "../models/LSTM_notewise.pth"

#model = loadStateDict(model, pathToModel)


In [10]:
def train(epoch):
    
    model.train()
    train_loss = 0
    criterion = nn.CrossEntropyLoss()
    for batch_idx, data in enumerate(train_loader):
        optimizer.zero_grad()
        input_lstm, ground_truth, seq_lengths = reorderBatch(data)
        prediction = model(input_lstm.double().to(device), seq_lengths)
        ground_truth = ground_truth.to(device)
        
        #print('inptu_lstm: ', input_lstm.size())
        #print("prediction: ",prediction.size(),"ground_truths: ", ground_truth.size())
        #print('prediction', torch.argmax(prediction[0,:,:]))
        #print('ground_truth', torch.argmax(ground_truth[0,:,:]))
        
        loss = criterion(prediction, ground_truth.contiguous().view(-1).long())
        loss.backward()
        
        print("lstm gradients: ", model.lstm.weight_ih_l0.grad)

        train_loss += loss.item()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        #reshape
        prediction = prediction.view(model.batch_size,-1, num_classes, 2)
        #delete negative outputs
        prediction = prediction[:,:,:,0] 
        print("prediction: ",prediction.size(),"ground_truth: ", ground_truth.size())
        if(batch_idx % log_interval == 0):
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * input_lstm.size(0), len(train_loader.dataset),
                100. * batch_idx / len(train_loader),
                loss.item() / input_lstm.size(0)))
            print('ground_truth', torch.argmax(ground_truth[0], dim=1))
            print('prediction', torch.argmax(prediction[0], dim=1))
            
    print('====> Epoch: {} Average Loss: {:.4f}'.format(
          epoch, train_loss / len(train_loader.dataset)))
        
    return train_loss / len(train_loader.dataset)

def test(epoch):
    model.eval()
    test_loss = 0
    criterion = nn.CrossEntropyLoss()
    with torch.no_grad():
        for i, data in enumerate(test_loader):
            input_lstm, ground_truth, seq_lengths = reorderBatch(data)
            prediction = model(input_lstm.double().to(device), seq_lengths)
            test_loss += criterion(prediction, ground_truth.view(-1).long().to(device)).item()
            prediction = prediction.view(model.batch_size,-1,num_classes)
            if(False):#i % log_interval == 0):
                print('prediction', torch.argmax(prediction,dim=2))
                print('ground_truth', torch.argmax(ground_truth,dim=2))
    
    print('====> Test set Loss: {:.4f}'.format(test_loss/len(test_loader.dataset)))
    return test_loss/len(test_loader.dataset)

def val(epoch):
    model.eval()
    val_loss = 0
    criterion = nn.CrossEntropyLoss()
    with torch.no_grad():
        for i, data in enumerate(val_loader):
            input_lstm, ground_truth, seq_lengths = reorderBatch(data)
            prediction = model(input_lstm.double().to(device), seq_lengths)
            val_loss += criterion(prediction, ground_truth.to(device)).item()
            
            break
    print('====> Validation Loss: {:.4f}'.format(val_loss))
    return val_loss
        

In [11]:
import matplotlib.pyplot as plt


train_losses = []
test_losses = []
if validate:
    val_losses = []
best_test_loss = 999
for epoch in range(1, epochs + 1):
    #train
    current_train_loss = (train(epoch))
    train_losses.append(current_train_loss)
    #test
    current_test_loss = test(epoch)
    test_losses.append(current_test_loss)
    #save if model better than best model
    if(current_test_loss < best_test_loss):
        best_test_loss = current_test_loss
        #torch.save(model.state_dict(),'/media/EXTHD/niciData/LSTM_notewise.pth')
    
    #validate
    if validate:
        current_val_loss = val(epoch)
        val_losses.append(current_val_loss)
 
    
plt.plot(train_losses, color='red', label='Train loss')
plt.plot(test_losses, color='orange', label='Test loss')
if validate:
    plt.plot(val_losses, color='yellow', label='Validation loss')
plt.legend()
plt.savefig('LSTM_notewise_Nottingham.png')
#plt.show()

/home/nici/workspace/.env3/lib/python3.5/site-packages/pretty_midi/pretty_midi.py:100: RuntimeWarning: Tempo, Key or Time signature change events found on non-zero tracks.  This is not a valid type 0 or type 1 MIDI file.  Tempo, Key or Time Signature may be wrong.
  RuntimeWarning)


lstm gradients:  tensor([[ 8.0819e-05, -2.7150e-05,  4.6065e-05,  ...,  1.2552e-04,
         -1.9836e-04,  2.9426e-05],
        [ 8.4046e-05,  1.3505e-04,  1.1301e-04,  ...,  1.6413e-04,
          3.5838e-05,  1.6570e-05],
        [-8.0412e-05, -1.7736e-05, -7.4031e-05,  ...,  2.3825e-05,
          5.1795e-05, -1.3786e-05],
        ...,
        [-5.3285e-05,  9.8481e-05, -2.0244e-04,  ..., -2.2879e-04,
          2.4916e-04,  3.4715e-05],
        [ 9.4228e-06, -3.9444e-05,  1.5473e-04,  ...,  1.3631e-04,
         -5.8519e-05, -8.7096e-05],
        [-8.5752e-05,  1.0426e-05,  1.7060e-05,  ..., -5.5685e-05,
          1.4286e-04, -6.6517e-05]], device='cuda:0', dtype=torch.float64)
prediction:  torch.Size([7, 15, 61]) ground_truth:  torch.Size([7, 15, 61])
Train Epoch: 1 [0/694 (0%)]	Loss: 0.179624
ground_truth tensor([31, 60, 60, 38, 43, 60, 60, 35, 38, 60, 60, 60, 38, 38, 60], device='cuda:0')
prediction tensor([16, 35, 24, 55, 55, 23, 23, 55, 23, 55, 23, 23, 16, 40, 40], device='cuda:0'

lstm gradients:  tensor([[ 3.9123e-07, -1.7047e-06,  1.6164e-06,  ...,  5.8197e-07,
         -2.2746e-06,  1.0797e-07],
        [-4.7490e-06,  6.7280e-06, -2.3730e-06,  ..., -2.5603e-06,
          8.1819e-06, -1.1116e-06],
        [-7.1696e-06, -2.3546e-06, -1.0719e-05,  ..., -9.4640e-06,
          7.6076e-06,  2.6490e-06],
        ...,
        [-2.2609e-06,  1.5030e-06, -4.9950e-06,  ..., -6.7135e-06,
          4.1072e-06,  2.1926e-06],
        [-2.1208e-07, -1.0527e-07,  4.7642e-08,  ..., -4.0741e-07,
          2.8511e-07, -1.0502e-07],
        [-1.9082e-06,  1.2174e-06, -2.4236e-06,  ..., -4.3674e-06,
          3.9649e-06,  4.5412e-07]], device='cuda:0', dtype=torch.float64)
prediction:  torch.Size([7, 15, 61]) ground_truth:  torch.Size([7, 15, 61])
Train Epoch: 1 [70/694 (10%)]	Loss: 0.014770
ground_truth tensor([33, 60, 60, 60, 26, 60, 60, 33, 38, 60, 60, 38, 37, 60, 60], device='cuda:0')
prediction tensor([ 4, 48, 48, 12, 16, 21, 21, 16, 21, 21, 21, 16, 21, 16, 21], device='cuda:

lstm gradients:  tensor([[-2.4586e-07, -8.4294e-08,  9.1515e-07,  ..., -2.1931e-07,
         -6.0649e-07, -1.5005e-07],
        [-3.1400e-06,  1.2986e-06,  8.7187e-07,  ..., -1.8193e-06,
          3.4273e-06, -1.2875e-06],
        [-1.7524e-08,  5.7108e-07, -3.6643e-06,  ..., -1.4457e-06,
          2.6396e-06,  9.4960e-07],
        ...,
        [-3.8429e-07,  3.4390e-07, -2.8305e-07,  ..., -9.3757e-07,
         -9.2990e-07,  4.2536e-07],
        [-2.0343e-07,  4.9278e-07, -6.0369e-07,  ..., -2.1537e-07,
          9.8148e-07,  6.9104e-08],
        [ 1.0272e-06, -1.0531e-06,  1.8680e-06,  ...,  2.5294e-06,
         -3.4967e-06, -8.4689e-07]], device='cuda:0', dtype=torch.float64)
prediction:  torch.Size([7, 15, 61]) ground_truth:  torch.Size([7, 15, 61])
Train Epoch: 1 [140/694 (20%)]	Loss: 0.010458
ground_truth tensor([31, 31, 60, 36, 36, 36, 60, 40, 40, 60, 60, 36, 36, 36, 60], device='cuda:0')
prediction tensor([54, 54, 48, 54, 16, 21, 21, 21, 21, 21, 16, 16, 16, 16, 16], device='cuda

lstm gradients:  tensor([[-1.7921e-08, -2.0804e-07,  9.9322e-08,  ..., -5.2761e-08,
         -6.3379e-08, -1.3097e-07],
        [ 1.8206e-06, -2.4912e-06,  2.4416e-06,  ...,  4.5787e-06,
         -4.4383e-06, -3.5893e-07],
        [-5.6997e-07,  4.6348e-07, -1.4257e-06,  ..., -1.6508e-06,
          1.1958e-06,  7.7542e-07],
        ...,
        [-4.1822e-07,  3.5178e-07, -9.4990e-07,  ..., -1.4248e-06,
         -2.9212e-07,  8.9244e-07],
        [-2.9277e-07,  1.2853e-07, -1.4554e-07,  ..., -4.6141e-07,
          2.1446e-07,  8.3924e-08],
        [-4.3777e-07, -1.9967e-07, -1.1380e-07,  ..., -1.1009e-06,
          5.4831e-07, -3.2448e-07]], device='cuda:0', dtype=torch.float64)
prediction:  torch.Size([7, 15, 61]) ground_truth:  torch.Size([7, 15, 61])
Train Epoch: 1 [210/694 (30%)]	Loss: 0.006912
ground_truth tensor([38, 60, 60, 35, 31, 60, 60, 31, 31, 60, 60, 38, 38, 38, 38], device='cuda:0')
prediction tensor([11, 48, 55, 12, 12, 12, 16, 16, 16, 16, 16, 16, 16, 16, 16], device='cuda

...

lstm gradients:  tensor([[-4.8304e-07, -4.9197e-07,  4.0247e-07,  ..., -1.2777e-06,
          4.7887e-08,  1.5044e-07],
        [-7.3751e-07, -1.7043e-06,  3.4131e-06,  ...,  2.2128e-07,
          1.3606e-06, -1.8527e-06],
        [ 9.0554e-07, -1.5975e-06,  2.6897e-06,  ...,  5.6280e-07,
         -1.5156e-06, -1.9755e-07],
        ...,
        [-7.2822e-07, -1.0329e-06,  7.8401e-07,  ..., -7.4687e-07,
          7.5186e-07, -2.7857e-06],
        [ 4.3225e-07,  1.3662e-06, -1.0526e-06,  ..., -7.6314e-07,
          9.5280e-07,  8.6216e-07],
        [ 1.5963e-07,  6.3840e-07, -9.4960e-07,  ...,  1.1907e-06,
          1.9026e-07,  1.3892e-07]], device='cuda:0', dtype=torch.float64)
prediction:  torch.Size([7, 15, 61]) ground_truth:  torch.Size([7, 15, 61])
Train Epoch: 3 [259/694 (37%)]	Loss: 0.004688
ground_truth tensor([35, 60, 60, 38, 38, 38, 60, 37, 60, 38, 60, 40, 40, 40, 60], device='cuda:0')
prediction tensor([ 4, 55,  4, 55, 51, 12, 11, 55, 18, 12, 12, 11, 18, 11, 12], device='cuda

lstm gradients:  tensor([[-4.8176e-07,  1.0695e-07, -6.8181e-07,  ..., -1.1170e-06,
          1.4128e-06, -1.2588e-07],
        [ 4.3213e-06, -2.1371e-06,  6.9570e-06,  ...,  4.3757e-06,
         -3.9126e-06, -9.5562e-07],
        [-2.2051e-06, -3.0320e-07, -1.1367e-06,  ..., -3.0978e-06,
          2.9311e-06, -6.2137e-07],
        ...,
        [ 3.2899e-08,  1.5333e-06, -4.2594e-06,  ..., -3.2903e-06,
          2.3999e-06,  7.4019e-07],
        [ 1.0975e-07, -2.7174e-07,  1.0328e-06,  ...,  6.4487e-07,
         -1.3670e-06, -2.3751e-07],
        [ 5.9389e-07, -2.9293e-07,  5.4615e-07,  ...,  1.2914e-06,
         -1.7171e-06, -1.6564e-07]], device='cuda:0', dtype=torch.float64)
prediction:  torch.Size([7, 15, 61]) ground_truth:  torch.Size([7, 15, 61])
Train Epoch: 3 [329/694 (47%)]	Loss: 0.004392
ground_truth tensor([31, 31, 60, 30, 30, 30, 30, 30, 30, 30, 60, 26, 26, 26, 60], device='cuda:0')
prediction tensor([54, 54, 55, 55, 55, 55, 16, 16, 16, 55, 16, 55, 16, 55, 16], device='cuda

lstm gradients:  tensor([[ 3.7912e-07,  7.6954e-08,  3.6066e-07,  ...,  6.1020e-07,
         -6.0764e-07, -1.2608e-07],
        [ 5.7318e-06, -1.5198e-06,  8.2932e-06,  ...,  1.2577e-05,
         -9.5127e-06, -2.4341e-06],
        [ 7.1338e-08,  8.1296e-07,  2.8072e-07,  ...,  1.4660e-06,
          2.7863e-07,  5.2776e-07],
        ...,
        [-9.5064e-07,  2.7839e-06, -1.2430e-06,  ..., -4.1412e-06,
          4.2798e-06,  1.6063e-07],
        [ 7.2027e-07, -4.2871e-07, -2.8296e-07,  ..., -4.7438e-07,
         -2.5716e-07, -1.6288e-07],
        [-1.9615e-06,  1.0089e-06, -4.8352e-06,  ..., -6.3028e-06,
          4.3391e-06,  1.1165e-06]], device='cuda:0', dtype=torch.float64)
prediction:  torch.Size([7, 15, 61]) ground_truth:  torch.Size([7, 15, 61])
Train Epoch: 3 [399/694 (58%)]	Loss: 0.005200
ground_truth tensor([38, 60, 60, 30, 30, 60, 60, 31, 31, 60, 60, 33, 35, 60, 60], device='cuda:0')
prediction tensor([ 4, 54, 55, 55, 55, 18, 16, 16, 16, 18, 16, 16, 16, 18, 16], device='cuda

lstm gradients:  tensor([[ 1.6246e-06,  7.6421e-07,  5.0113e-07,  ...,  2.0325e-06,
         -5.5834e-07, -1.8630e-07],
        [ 2.9227e-06, -2.6960e-06,  6.9009e-06,  ...,  1.1752e-05,
         -1.2017e-05, -9.2276e-07],
        [ 5.1029e-08, -2.7251e-08,  3.4625e-07,  ...,  1.6246e-07,
          9.2069e-07, -5.3790e-07],
        ...,
        [ 2.2233e-06, -8.2143e-07,  2.4063e-06,  ...,  7.4569e-06,
         -7.7073e-06,  2.0097e-06],
        [-1.8013e-06,  9.7677e-07, -2.7771e-06,  ..., -3.8375e-06,
          5.2003e-06, -6.6662e-07],
        [-1.5044e-06, -1.2986e-07, -1.2904e-06,  ..., -2.8278e-06,
          3.5366e-06, -5.1850e-07]], device='cuda:0', dtype=torch.float64)
prediction:  torch.Size([7, 15, 61]) ground_truth:  torch.Size([7, 15, 61])
Train Epoch: 3 [469/694 (68%)]	Loss: 0.005061
ground_truth tensor([60, 28, 28, 60, 33, 33, 60, 60, 60, 60, 60, 60, 33, 33, 60], device='cuda:0')
prediction tensor([ 4, 18, 18, 16, 16,  0, 16, 55, 16,  0, 18, 16, 16,  0, 16], device='cuda

KeyboardInterrupt: 

# Generate

In [ ]:
np.set_printoptions(threshold=np.inf, suppress=1)


In [ ]:

model.eval()
model.batch_size=1
with torch.no_grad():
    #get unseeen sample from validation set
    for data in val_loader:
        input_lstm, ground_truth, seq_lengths = reorderBatch(data)
        print(input_lstm.size())
        prediction = model(input_lstm.double().to(device), seq_lengths)
        print(prediction.size())
        


        prediction = prediction.squeeze(0).cpu().numpy()
        #batch size must be 1 for generation
        prediction = prediction[:,0:1].reshape(input_lstm.size(1),input_lstm.size(2))
        
        print(prediction.shape)

        #normalize to [0,1]
        pred_max = np.max(prediction)
        pred_min = np.min(prediction)
        prediction = (prediction - pred_min) / (pred_max - pred_min)
        #print(prediction)
        
        prediction[prediction < 0.7] = 0
        
        prediction = debinarizeMidi(prediction, prediction=True)
        prediction = addCuttedOctaves(prediction)
        #print(prediction)
        pianorollMatrixToTempMidi(prediction, show=True, showPlayer=True, autoplay=True,
                                 path='../temp/temp.mid')
        

print('')